<a href="https://colab.research.google.com/github/minassy/ISID_2021/blob/main/210904_%E5%AD%A6%E7%BF%92%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#データ準備

In [2]:
import os
import zipfile
import glob
import pandas as pd
import numpy as np
import pathlib
from tqdm import tqdm

In [3]:
#作業基本ディレクトリ
DIR = 'drive/MyDrive/00_Competition/ISID/2021'
os.listdir(DIR)
#os.mkdir(DIR+'/data')

['210904.ipynb', 'data']

In [4]:
os.listdir(os.path.join(DIR, 'data'))

['ISID_08_09_2021.csv',
 'Test_Unit_20210903.zip',
 'Train_Unit_20210903.zip',
 'Train_Unit_20210903',
 'Test_Unit_20210903']

In [5]:
#データセットの解凍
#TARGET_PATH_train = os.path.join(DIR, 'data', 'Train_Unit_20210903.zip')
#TARGET_PATH_test = os.path.join(DIR, 'data', 'Test_Unit_20210903.zip')
#OUT_PATH = os.path.join(DIR, 'data')
#with zipfile.ZipFile(TARGET_PATH_test, 'r') as f: 
#  f.extractall(path=OUT_PATH)

In [6]:
DIR_DATA_TRAIN = os.path.join(DIR, 'data', 'Train_Unit_20210903')
DIR_DATA_TEST = os.path.join(DIR, 'data', 'Test_Unit_20210903')

In [7]:
print(os.listdir(DIR_DATA_TRAIN))
print(os.listdir(DIR_DATA_TRAIN + '/Train_Unit_1'))

['Train_Unit_3', 'Train_Unit_2', 'Train_Unit_1']
['Charge', 'Discharge']


In [8]:
#サブフォルダを含む、全ファイルを取得しファイル数を確認。
#訓練データ
for folder in os.listdir(DIR_DATA_TRAIN):
  for C in os.listdir(os.path.join(DIR_DATA_TRAIN, folder)):
    files = glob.glob(os.path.join(DIR_DATA_TRAIN, folder, C, '*.csv'))
    print(f'{folder}','--',f'{C}のCSVファイル数：',len(files))

Train_Unit_3 -- ChargeのCSVファイル数： 97
Train_Unit_3 -- DischargeのCSVファイル数： 97
Train_Unit_2 -- ChargeのCSVファイル数： 40
Train_Unit_2 -- DischargeのCSVファイル数： 40
Train_Unit_1 -- ChargeのCSVファイル数： 124
Train_Unit_1 -- DischargeのCSVファイル数： 124


In [9]:
#サブフォルダを含む、全ファイルを取得しファイル数を確認。
#テストデータ
for folder in os.listdir(DIR_DATA_TEST):
  for C in os.listdir(os.path.join(DIR_DATA_TEST, folder)):
    files = glob.glob(os.path.join(DIR_DATA_TEST, folder, C, '*.csv'))
    print(f'{folder}','--',f'{C}のCSVファイル数：',len(files))

Test_Unit_1 -- ChargeのCSVファイル数： 70
Test_Unit_1 -- DischargeのCSVファイル数： 70
Test_Unit_2 -- ChargeのCSVファイル数： 12
Test_Unit_2 -- DischargeのCSVファイル数： 12
Test_Unit_3 -- ChargeのCSVファイル数： 55
Test_Unit_3 -- DischargeのCSVファイル数： 55


In [10]:
#データ例の確認
folder = 'Train_Unit_1'
C = 'Charge'
exp_files = glob.glob(os.path.join(DIR_DATA_TRAIN, folder, C, '*.csv'))
exp_file = [s for s in exp_files if 'File_1.csv' in s]

In [11]:
exp_file[0]

'drive/MyDrive/00_Competition/ISID/2021/data/Train_Unit_20210903/Train_Unit_1/Charge/Training_Unit_1_File_1.csv'

In [12]:
tmp_df = pd.read_csv(exp_file[0], encoding='utf-8')

In [13]:
#データの確認
Dataframe = tmp_df
print('レコード数：',Dataframe.shape[0],'カラム数：',Dataframe.shape[1])
print('/'*30)
print('データセット情報:')
Dataframe.info()
print('/'*30)
print('欠損値確認：')
print(Dataframe.isnull().sum())
print('/'*30)
Dataframe.head()

レコード数： 940 カラム数： 7
//////////////////////////////
データセット情報:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Time (s)                 940 non-null    float64
 1   Voltage Measured (V)     940 non-null    float64
 2   Voltage Charge (V)       940 non-null    float64
 3   Current Measured (Amps)  940 non-null    float64
 4   Current Charge (Amps)    940 non-null    float64
 5   Temperature (degree C)   940 non-null    float64
 6   Unnamed: 6               0 non-null      float64
dtypes: float64(7)
memory usage: 51.5 KB
//////////////////////////////
欠損値確認：
Time (s)                     0
Voltage Measured (V)         0
Voltage Charge (V)           0
Current Measured (Amps)      0
Current Charge (Amps)        0
Temperature (degree C)       0
Unnamed: 6                 940
dtype: int64
//////////////////////////////


,Time (s),Voltage Measured (V),Voltage Charge (V),Current Measured (Amps),Current Charge (Amps),Temperature (degree C),Unnamed: 6
0,0.000,3.3251,0.003,0.000302,-0.002,29.342,NaN
1,2.516,3.0020,1.414,-3.362000,-3.370,29.336,NaN
2,5.500,3.4346,4.142,1.508700,1.498,29.335,NaN
3,8.391,3.4549,4.180,1.510000,1.498,29.342,NaN
4,11.266,3.4688,4.196,1.508700,1.498,29.331,NaN


メモ(現時点での分析方針のアイデア)  
■どういうデータセット構成が必要かを考える。  
■ChargeとDischargeについて、サイクル毎で統合が必要か？  
  ⇒それぞれが、残寿命に対する予測に寄与する特徴量となる？  
  ⇒ChargeとDischaegeをつなぐキー列は何？(候補：サイクル数 + Time(s)か？)  
  ⇒たぶん違う。  
  ⇒ChargeモデルとDischargeモデルがおそらく必要。  
  ⇒それぞれのモデルから出された予測について、アンサンブルする？(平均 or 重み付け)  
  ■目的変数が"サイクル数"なので、ファイル名からサイクル数の付与が必要。  
■1ファイルにつき、1統計量？みたいな感じで、データの丸めが必要か。  
■EDA的に、サイクル数と特徴量の時系列変化を見て、サイクル数増える(命を削る)ことに対し、増減変動している特徴は何か？を確認する。  
■ChargeとDischargeの情報を縮約して、予測する。  
■モデル数としては、全部で6つ。  
ユニット1の充電と放電、ユニット2の充電と放電、ユニット3の充電と放電。  
⇒テストに使うモデルの選択は、データの傾向を見て、同じようなものを予測モデルに適用する流れか？  
■打ち切りのデータ。ある意味、機械学習というよりは信頼工学的なアプローチを取ってもいいかもしれない。

# データ処理

・サイクル数の列を付与する。

In [14]:
#expファイルで試す。
tmp_df.head()

,Time (s),Voltage Measured (V),Voltage Charge (V),Current Measured (Amps),Current Charge (Amps),Temperature (degree C),Unnamed: 6
0,0.000,3.3251,0.003,0.000302,-0.002,29.342,NaN
1,2.516,3.0020,1.414,-3.362000,-3.370,29.336,NaN
2,5.500,3.4346,4.142,1.508700,1.498,29.335,NaN
3,8.391,3.4549,4.180,1.510000,1.498,29.342,NaN
4,11.266,3.4688,4.196,1.508700,1.498,29.331,NaN


In [15]:
#フォルダ名から、ユニット名を取得。
dirname = os.path.dirname(exp_file[0])
dirname_split = dirname.split('/')#区切り文字'/'で分割
dirname_split
unit = dirname_split[-2]
charge_mode = dirname_split[-1]
unit, charge_mode

('Train_Unit_1', 'Charge')

In [16]:
#ファイル名から、サイクル数を取得。
basename = os.path.basename(exp_file[0])
basename_split = basename.split('_')[4]#区切り文字'_'で分割
cycle_num = basename_split.split('.')[0]
cycle_num

'1'

In [17]:
#新たに取得した、ユニット名、充・放電モード、サイクル数を付与
tmp_df[['unit', 'charge_mode', 'Cycle_num']] = unit, charge_mode, cycle_num
tmp_df.head()

,Time (s),Voltage Measured (V),Voltage Charge (V),Current Measured (Amps),Current Charge (Amps),Temperature (degree C),Unnamed: 6,unit,charge_mode,Cycle_num
0,0.000,3.3251,0.003,0.000302,-0.002,29.342,NaN,Train_Unit_1,Charge,1
1,2.516,3.0020,1.414,-3.362000,-3.370,29.336,NaN,Train_Unit_1,Charge,1
2,5.500,3.4346,4.142,1.508700,1.498,29.335,NaN,Train_Unit_1,Charge,1
3,8.391,3.4549,4.180,1.510000,1.498,29.342,NaN,Train_Unit_1,Charge,1
4,11.266,3.4688,4.196,1.508700,1.498,29.331,NaN,Train_Unit_1,Charge,1


In [18]:
#一連の流れを関数化。
def get_some_values(file_path):
  #ファイルパスから、ユニット名を取得。
  dirname = os.path.dirname(file_path)
  dirname_split = dirname.split('/')#区切り文字'/'で分割
  unit = dirname_split[-2]
  charge_mode = dirname_split[-1]
  #ファイルパスから、サイクル数を取得。
  basename = os.path.basename(file_path)
  basename_split = basename.split('_')[4]#区切り文字'_'で分割
  cycle_num = basename_split.split('.')[0]
  return unit, charge_mode, cycle_num

全ユニットの全フォルダに対して連続処理し、1つのデータセットとして保有するためのデータ連結処理

In [20]:
%time
df_list = []
for folder in tqdm(os.listdir(DIR_DATA_TRAIN)):
  df_list_1 = []
  for C in ['Charge', 'Discharge']:
    files = glob.glob(os.path.join(DIR_DATA_TRAIN, folder, C, '*.csv'))
    df_list_0 = []
    for file in files:
      tmp_df = pd.read_csv(file, encoding='utf-8')
      unit, charge_mode, cycle_num = get_some_values(file)
      #新たに取得した、ユニット名、充・放電モード、サイクル数を付与
      tmp_df[['unit', 'charge_mode', 'Cycle_num']] = unit, charge_mode, cycle_num
      df_list_0.append(tmp_df)
    df_0 = pd.concat(df_list_0)
    df_list_1.append(df_0)
  df_1 = pd.concat(df_list_1)
  df_list.append(df_1)
df = pd.concat(df_list)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


100%|██████████| 3/3 [01:21<00:00, 27.13s/it]


In [21]:
df.shape

(756089, 12)

In [22]:
df[['Cycle_num', 'unit', 'charge_mode']].nunique()

Cycle_num      124
unit             3
charge_mode      2
dtype: int64

In [23]:
df.head()

,Time (s),Voltage Measured (V),Voltage Charge (V),Current Measured (Amps),Current Charge (Amps),Temperature (degree C),Unnamed: 6,unit,charge_mode,Cycle_num,Voltage Load (V),Current Load (Amps)
0,0.000,3.6927,0.003,-0.004495,0.000,25.663,NaN,Train_Unit_3,Charge,90,NaN,NaN
1,2.453,3.1721,1.462,-3.573100,-3.563,25.660,NaN,Train_Unit_3,Charge,90,NaN,NaN
2,8.921,3.8846,4.618,1.505800,1.498,25.743,NaN,Train_Unit_3,Charge,90,NaN,NaN
3,15.156,3.9054,4.650,1.506600,1.500,25.835,NaN,Train_Unit_3,Charge,90,NaN,NaN
4,21.750,3.9165,4.661,1.509000,1.500,25.884,NaN,Train_Unit_3,Charge,90,NaN,NaN


In [24]:
#データの確認
Dataframe = df
print('レコード数：',Dataframe.shape[0],'カラム数：',Dataframe.shape[1])
print('/'*30)
print('データセット情報:')
Dataframe.info()
print('/'*30)
print('欠損値確認：')
print(Dataframe.isnull().sum())
print('/'*30)
Dataframe.head()

レコード数： 756089 カラム数： 12
//////////////////////////////
データセット情報:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 756089 entries, 0 to 300
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Time (s)                 756089 non-null  float64
 1   Voltage Measured (V)     756088 non-null  float64
 2   Voltage Charge (V)       699412 non-null  float64
 3   Current Measured (Amps)  756088 non-null  float64
 4   Current Charge (Amps)    699412 non-null  float64
 5   Temperature (degree C)   756088 non-null  float64
 6   Unnamed: 6               0 non-null       float64
 7   unit                     756089 non-null  object 
 8   charge_mode              756089 non-null  object 
 9   Cycle_num                756089 non-null  object 
 10  Voltage Load (V)         56677 non-null   float64
 11  Current Load (Amps)      56677 non-null   float64
dtypes: float64(9), object(3)
memory usage: 75.0+ MB
/////

,Time (s),Voltage Measured (V),Voltage Charge (V),Current Measured (Amps),Current Charge (Amps),Temperature (degree C),Unnamed: 6,unit,charge_mode,Cycle_num,Voltage Load (V),Current Load (Amps)
0,0.000,3.6927,0.003,-0.004495,0.000,25.663,NaN,Train_Unit_3,Charge,90,NaN,NaN
1,2.453,3.1721,1.462,-3.573100,-3.563,25.660,NaN,Train_Unit_3,Charge,90,NaN,NaN
2,8.921,3.8846,4.618,1.505800,1.498,25.743,NaN,Train_Unit_3,Charge,90,NaN,NaN
3,15.156,3.9054,4.650,1.506600,1.500,25.835,NaN,Train_Unit_3,Charge,90,NaN,NaN
4,21.750,3.9165,4.661,1.509000,1.500,25.884,NaN,Train_Unit_3,Charge,90,NaN,NaN


In [25]:
#CSVファイルとして一旦出力
df.to_csv(os.path.join(DIR_DATA_TRAIN, 'dataset_train_all.csv'), encoding='utf-8')

In [27]:
os.listdir(DIR_DATA_TRAIN)

['Train_Unit_3', 'Train_Unit_2', 'Train_Unit_1', 'dataset_train_all.csv']